Cartar de posible desvio para la union

In [1]:
import sys
sys.path.append('..')

import pandas as pd
import collections
import os

from docxtpl import DocxTemplate

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [2]:
RUTA_COMPLETA

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [3]:
path_xlsx_avance = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - AVANCE COSECHA\2025\AVANCE SEMANAL\AVANCE DE COSECHA V1.xlsx'

In [4]:
xlsx_entregas = pd.read_excel(path_xlsx_avance, sheet_name='DETALLE ENTREGAS')
xlsx_avance = pd.read_excel(path_xlsx_avance, sheet_name = 'AVANCE')

In [5]:
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas = xlsx_entregas.drop(xlsx_entregas.index[-1])
xlsx_entregas['cod_cos'] = xlsx_entregas['cod_cos'].astype(int)
xlsx_entregas['cod_ca'] = xlsx_entregas['cod_ca'].astype(int)
xlsx_entregas['%ENTREGA'] = xlsx_entregas['%ENTREGA'] * 100
xlsx_entregas.head(3)

,inst,cod_cos,cod_ca,nom_ca,TOTAL COMPROMISO,TOTAL ENTREGAS,%ENTREGA
0,1,49,660,AGUILERA ANTELO ALEX ENRIQUE,982.61,627.21,63.831021
1,1,54,3361,ENRIQUEZ ROJAS ALBERTO,1615.39,1460.25,90.396127
2,1,55,3460,ESPINOZA MENDOZA SANTIAGO,5288.56,6138.79,116.076777


In [6]:
xlsx_avance.head(3)

,COD COS,INST,COD CA,NOMBRE,COMPROMISO,ENTREGAS,%ENTREGA,TOTAL AREA,COSECHADO,SIN COSECHAR,...,TCH,%DIFF AVANCE,TCH EST,PRODUC EST,EST PRODUC COSECHADO,DIF. EST ENTREGA,%DIF. EST ENTREGA,TN EN CAMPO,COMPROMISO RES,DIFERENCIA
0,1,50,515,AGUILERA OLGA RIVERO VDA DE,30572.11,24584.36,0.804143,618.27,393.00,219.30,...,62.555623,0.168499,57.639599,35292.15,22949.92,1634.44,0.933517,12342.23,5987.75,6354.48
1,2,69,1180,BALCAZAR BALCAZAR JESUS E.,10965.29,11870.94,1.082592,200.94,195.54,3.34,...,60.708500,0.109466,53.082819,10569.32,10385.82,1485.12,0.874894,183.50,-905.65,1089.15
2,3,69,911,BALCAZAR JUSTINIANO MARIO,5422.67,3339.39,0.615820,122.97,60.95,61.37,...,54.789007,0.120171,51.431188,6289.52,3275.12,64.27,0.980754,3014.40,2083.28,931.12


In [7]:
xlsx_avance.columns

Index(['COD COS', 'INST ', 'COD CA', 'NOMBRE', 'COMPROMISO', 'ENTREGAS',
       '%ENTREGA', 'TOTAL AREA', 'COSECHADO', 'SIN COSECHAR', 'SEMILLA',
       '%COSECHADO', 'TCH', '%DIFF AVANCE', 'TCH EST', 'PRODUC EST',
       'EST PRODUC COSECHADO', 'DIF. EST ENTREGA', '%DIF. EST ENTREGA',
       'TN EN CAMPO', 'COMPROMISO RES ', 'DIFERENCIA'],
      dtype='object')

In [9]:
### fecha_inf = '17 de octubre de 2025'
fecha_form = '17-10-2025'

fecha_img = '17 de octubre de 2025'

cod_grupos = [327]

In [10]:
def crear_reporte(lista_cod_cos):
    for cod in lista_cod_cos:
        grupo = xlsx_entregas[xlsx_entregas['cod_cos'] == cod].copy()
        
        # convertir datos del grupo a objetos
        list_obj_caneros = []
        for index, row in grupo.iterrows():
            canero = {
            'inst' : row['inst'],
            'cod_ca' : row['cod_ca'],
            'nom_ca' : row['nom_ca'],
            'compromiso' : row['TOTAL COMPROMISO'],
            'entrega' : row['TOTAL ENTREGAS'],
            'entrega_porcen' : row['%ENTREGA']
            }
            list_obj_caneros.append(collections.namedtuple("canero", canero.keys())(*canero.values()))
        # totales de los datos del grupo
        grupo_totales = {
            'compromiso' : grupo['TOTAL COMPROMISO'].sum(),
            'entregas' : grupo['TOTAL ENTREGAS'].sum(),
            'entrega_porcen' : grupo['TOTAL ENTREGAS'].sum() / grupo['TOTAL COMPROMISO'].sum() * 100
        }
        obj_grupo_totales = collections.namedtuple("grupo_totales", grupo_totales.keys())(*grupo_totales.values())
        
        # datos generales de avance de cosecha
        avance = xlsx_avance[xlsx_avance['COD COS'] ==cod]
        avance_totales = {
            'fecha_img' : fecha_img,
            'fecha_informe' : fecha_inf,
            'compromiso' : float(avance['COMPROMISO'].iloc[0]),
            'entrega' : float(avance['ENTREGAS'].iloc[0]),
            'entrega_porcen' : float(avance['%ENTREGA'].iloc[0]) * 100,
            'ha_canha' : float(avance['COSECHADO'].iloc[0] + avance['SIN COSECHAR'].iloc[0]),
            'ha_cosecha' : float(avance['COSECHADO'].iloc[0]),
            'cosecha_porcen' : float(avance['%COSECHADO'].iloc[0]) * 100,
            'tch' : float(avance['TCH'].iloc[0])
        }
        # datos generales a objeto
        obj_avance_totales = collections.namedtuple("avance_totales",avance_totales.keys())(*avance_totales.values())
        
        # generar reporte
        doc = DocxTemplate(RUTA_COMPLETA + '/templates/tpl_rpt_avance_aclarativa_desvio.docx')
        context = {
            'insp' : list_obj_caneros,
            'grupo_totales' : obj_grupo_totales,
            'avance' : obj_avance_totales
        }
        doc.render(context)

        #REPROTE POSIBLE COMPRA DE CAÑA
        doc.save(RUTA_COMPLETA + '/informes/' + str(avance['COD CA'].iloc[0]) + '_RPCC_' + fecha_form + '_' + avance['NOMBRE'].iloc[0] + '.docx')
        print('Se generó reporte de grupo:', cod, ' - ', avance['NOMBRE'].iloc[0])

In [11]:
crear_reporte(cod_grupos)

Se generó reporte de grupo: 327  -  LAURA CAMACHO JACINTO
